In [21]:
import os
os.chdir('../') 

import pandas as pd
import numpy as np
from util import *

In [5]:
articles_df, interactions_full_df, \
    interactions_train_df, interactions_test_df, \
      cf_preds_df = load_dataset('data/')

In [15]:
user_games = interactions_train_df.groupby('user_id')['game'].apply(list).reset_index()
user_games.head()

,user_id,game
0,0,"[Day of Defeat, Half-Life, Counter-Strike Sour..."
1,1,"[Ricochet, The Stanley Parable, Thief Gold, To..."
2,2,"[Deus Ex Game of the Year Edition, Age of Empi..."
3,3,"[Half-Life Opposing Force, Team Fortress Class..."
4,4,"[Half-Life, Day of Defeat, Half-Life Opposing ..."


In [49]:
user_games.shape

(3757, 3)

In [35]:
def split_str(val):
    tokens = []
    for v in val:
        tokens.extend(v.split(' '))
    return " ".join(tokens)

In [36]:
user_games['tokens'] = user_games['game'].apply(split_str)
user_games.head(2)

,user_id,game,tokens
0,0,"[Day of Defeat, Half-Life, Counter-Strike Sour...",Day of Defeat Half-Life Counter-Strike Source ...
1,1,"[Ricochet, The Stanley Parable, Thief Gold, To...",Ricochet The Stanley Parable Thief Gold Total ...


In [55]:
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences

In [56]:
# integer encode the documents
vocab_size   = 100
encoded_docs = [one_hot(d, vocab_size) for d in user_games.tokens]
print(encoded_docs)

[[16, 33, 74, 79, 97, 58, 19, 63, 79, 97, 21, 75, 47, 76, 30, 54, 81, 56, 74, 19, 58, 55, 58, 19, 97, 79, 97, 21, 71, 59, 79, 97, 21, 55, 83, 72, 21, 75, 56, 79, 97, 21, 55, 89, 51, 49, 47, 21], [97, 45, 81, 34, 46, 79, 21, 35, 59, 46, 21, 46, 95, 50, 58, 19, 79, 97, 2, 33, 4, 94, 16, 35, 52, 59, 58, 19, 98, 21, 34, 21, 82, 83, 14, 34, 99, 33, 36, 51, 71, 34, 21, 82, 83, 46, 69, 34, 21, 46, 27, 46, 2, 33, 4, 7, 84, 86, 76, 16, 33, 74, 2, 33, 4, 7, 84, 21, 76, 54, 81, 56, 76, 30, 2, 33, 4, 7, 84, 86, 58, 19, 63, 46, 54, 91, 33, 63, 9, 4, 71, 46, 45, 66, 72, 2, 33, 4, 7, 84, 21], [74, 19, 45, 33, 45, 90, 71, 91, 33, 63, 9, 4, 45, 45, 91, 71, 27, 52, 20, 98, 79, 97, 21, 55, 83, 74, 19, 58, 55, 45, 49, 48, 49, 42, 21, 49, 42, 91, 33, 63, 9, 4, 71, 16, 33, 74, 45, 38, 77, 1, 53, 12, 61, 28, 52, 86, 5, 82, 7, 60, 69, 91, 71, 27, 52, 42, 21, 21, 45, 60, 10, 75, 56, 42, 21, 61, 28, 52, 19, 45, 41, 38, 55, 3, 43, 45, 55, 65, 79, 97, 21, 53, 1, 5, 46, 50, 62, 9, 92, 45, 89, 29, 18, 62, 45, 20, 5

In [57]:
# pad documents to a max length of 4 words
max_length  = 5
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs)

[[89 51 49 47 21]
 [33  4  7 84 21]
 [45 83 46 79 97]
 ...
 [70 78 97 72 21]
 [ 7 94 65  3 12]
 [21 84 51 45 45]]


In [58]:
padded_docs.shape

(3757, 5)

In [62]:
from keras.models import Sequential
from keras.layers import Input, Dense, Embedding, Flatten, Dropout, merge, Activation


# define the model
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=max_length))
#model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
# summarize the model
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 5, 10)             1000      
_________________________________________________________________
dense_4 (Dense)              (None, 5, 1)              11        
Total params: 1,011
Trainable params: 1,011
Non-trainable params: 0
_________________________________________________________________
None


In [60]:

# fit the model
model.fit(padded_docs, labels, epochs=50, verbose=0)
# evaluate the model
loss, accuracy = model.evaluate(padded_docs, labels, verbose=0)
print('Accuracy: %f' % (accuracy*100))

NameError: name 'labels' is not defined

In [22]:
np.reshape(user_games.game[0], -1)

array(['Day of Defeat', 'Half-Life', 'Counter-Strike Source',
       'Half-Life 2 Deathmatch', 'Portal', 'Alien Swarm',
       'Team Fortress Classic', 'Deus Ex Human Revolution',
       'Counter-Strike', 'Ricochet', 'Half-Life 2 Lost Coast',
       'Half-Life 2 Episode One', 'Dota 2', 'Deathmatch Classic',
       'Half-Life 2 Episode Two', 'Cities Skylines', 'Portal 2'],
      dtype='<U24')

In [25]:
user_games.game[0][0].split(" ")

['Day', 'of', 'Defeat']